<a href="https://colab.research.google.com/github/brennanMosher/False-Data-Injection-Detection-for-Smart-Grid-Systems/blob/main/py/all_feature_selection_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/brennanMosher/False-Data-Injection-Detection-for-Smart-Grid-Systems.git

Cloning into 'False-Data-Injection-Detection-for-Smart-Grid-Systems'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 164 (delta 73), reused 109 (delta 34), pack-reused 0
Receiving objects: 100% (164/164), 45.00 MiB | 13.88 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Checking out files: 100% (35/35), done.


In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SequentialFeatureSelector as sfs
from sklearn import utils
from matplotlib import pyplot

In [4]:
training_dataset = pd.read_csv('training_data_injection_and_normal_events_dataset.csv')
testing_dataset = pd.read_csv('testing_data_injection_and_normal_events_dataset.csv')


##Replace inf values with 0
training_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
testing_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
testing_dataset.head()
training_dataset.head()

,Unnamed: 0,R1-PA1:VH,R1-PM1:V,R1-PA2:VH,R1-PM2:V,R1-PA3:VH,R1-PM3:V,R1-PA4:IH,R1-PM4:I,R1-PA5:IH,...,control_panel_log4,relay1_log,relay2_log,relay3_log,relay4_log,snort_log1,snort_log2,snort_log3,snort_log4,marker
0,22122,151.008757,131534.3744,31.019935,130907.5427,-88.974616,131584.5210,144.551522,472.24069,26.510757,...,0,0,0,0,0,0,0,0,0,0
1,26934,152.246345,131910.4735,32.274713,131283.6417,-87.731298,131960.6200,151.071782,353.03608,31.518408,...,0,0,0,0,0,0,0,0,0,0
2,10625,-170.254409,131734.9606,69.780530,131709.8873,-50.225480,131810.1804,-169.830420,359.81115,67.706423,...,0,0,0,0,0,0,0,0,0,0
3,26778,-174.041660,134217.2143,65.987549,134192.1410,-54.007002,134292.4341,-174.293761,366.40311,70.015443,...,0,0,0,0,0,0,0,0,0,0
4,2558,39.058533,132361.7923,-80.924559,131709.8873,159.081732,132411.9389,46.690331,242.80386,-78.707212,...,0,0,0,0,0,0,0,0,0,1


In [6]:
training_dataset.fillna(training_dataset.mean(),inplace = True)
testing_dataset.fillna(testing_dataset.mean(),inplace = True)
X_train = training_dataset.iloc[:, 0:129].values
X_test = testing_dataset.iloc[:, 0:129].values
y_train = training_dataset.iloc[:, 129].values
y_test = testing_dataset.iloc[:, 129].values

In [7]:
def select_features(X_train, y_train, X_test):
	# configure to select all features
	fs = SelectKBest(score_func=f_classif, k='all')
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

In [8]:
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))

Feature 0: 37933.796895
Feature 1: 18.742091
Feature 2: 5.319716
Feature 3: 22.463856
Feature 4: 0.540175
Feature 5: 19.291128
Feature 6: 2.238262
Feature 7: 11.066391
Feature 8: 4.504486
Feature 9: 33.945025
Feature 10: 18.037743
Feature 11: 13.944467
Feature 12: 25.368003
Feature 13: 18.382092
Feature 14: 0.158732
Feature 15: 0.103416
Feature 16: 27.792456
Feature 17: 0.158192
Feature 18: 33.571655
Feature 19: 12.288876
Feature 20: 17.125342
Feature 21: 1.192143
Feature 22: 24.639312
Feature 23: 18.295912
Feature 24: 40.598990
Feature 25: 14.615636
Feature 26: 7.423261
Feature 27: 0.314475
Feature 28: 2.619174
Feature 29: 9.326779
Feature 30: 0.980861
Feature 31: 6.752633
Feature 32: 21.739620
Feature 33: 0.465386
Feature 34: 0.304185
Feature 35: 1.719914
Feature 36: 0.080334
Feature 37: 11.514209
Feature 38: 5.050085
Feature 39: 18.034152
Feature 40: 56.075879
Feature 41: 25.575688
Feature 42: 1.065385
Feature 43: 0.408970
Feature 44: 0.683686
Feature 45: 22.461978
Feature 46: 0.895

In [17]:
def select_features(X_train, y_train, X_test):
	# configure to select all features
	fs = SelectKBest(score_func=mutual_info_regression, k=10)
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs


In [21]:
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
#print(X_test_fs)
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))


Feature 0: 0.609039
Feature 1: 0.037981
Feature 2: 0.018262
Feature 3: 0.042186
Feature 4: 0.018838
Feature 5: 0.039716
Feature 6: 0.019635
Feature 7: 0.026188
Feature 8: 0.029182
Feature 9: 0.025897
Feature 10: 0.027870
Feature 11: 0.026680
Feature 12: 0.019559
Feature 13: 0.040391
Feature 14: 0.018554
Feature 15: 0.000251
Feature 16: 0.004727
Feature 17: 0.000000
Feature 18: 0.000000
Feature 19: 0.022760
Feature 20: 0.023989
Feature 21: 0.013853
Feature 22: 0.000906
Feature 23: 0.006803
Feature 24: 0.009440
Feature 25: 0.003703
Feature 26: 0.000000
Feature 27: 0.021364
Feature 28: 0.011149
Feature 29: 0.006847
Feature 30: 0.042754
Feature 31: 0.031960
Feature 32: 0.041810
Feature 33: 0.029934
Feature 34: 0.042380
Feature 35: 0.025404
Feature 36: 0.029599
Feature 37: 0.030801
Feature 38: 0.036731
Feature 39: 0.051178
Feature 40: 0.025127
Feature 41: 0.034427
Feature 42: 0.042269
Feature 43: 0.029628
Feature 44: 0.000000
Feature 45: 0.008301
Feature 46: 0.013594
Feature 47: 0.003876
Fe

In [25]:
clf = DecisionTreeClassifier()
sfs1 = sfs(clf)
sfs1.fit(X_train, y_train)

SequentialFeatureSelector(estimator=DecisionTreeClassifier())

In [62]:
sfs1.get_support()



array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,